#### Fully Convolutional Network

In [17]:
import torch
import torch.nn.functional as F

In [18]:
import torchvision.models as model
vgg=model.vgg16(weights=True)

In [19]:
from torchsummary import summary

In [20]:
summary(vgg,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [21]:
import torch.nn as nn

In [22]:
class FCN8(nn.Module):
    def __init__(self, num_classes=2):
        super(FCN8,self).__init__()
        vgg = model.vgg16(weights=True)

        self.features=vgg.features ## till convolutional and pooling

        self.pool3=self.features[:17]
        self.pool4=self.features[:24]
        self.pool5=self.features

        # 1x1 convolutions to convert to class scores
        self.conv1x1_3=nn.Conv2d(256,num_classes,kernel_size=1)
        self.conv1x1_4=nn.Conv2d(512,num_classes,kernel_size=1)
        self.conv1x1_5=nn.Conv2d(512,num_classes,kernel_size=1)  

        self.upsample32 = nn.ConvTranspose2d(num_classes,num_classes,kernel_size=4,stride=2,padding=1)
        self.upsample16 = nn.ConvTranspose2d(num_classes,num_classes,kernel_size=4,stride=2,padding=1)
        self.upsample8 = nn.ConvTranspose2d(num_classes,num_classes,kernel_size=16,stride=8,padding=4)

    def forward(self,x):
        pool3_out = self.pool3(x)
        pool4_out = self.pool4(x)
        pool5_out = self.pool5(x)

        pool3_out = self.conv1x1_3(pool3_out)
        pool4_out = self.conv1x1_4(pool4_out)
        pool5_out = self.conv1x1_5(pool5_out)

        x = self.upsample32(pool5_out) + pool4_out
        x = self.upsample16(x) + pool3_out
        x = self.upsample8(x)

        return x
        

###### I = 'Input Size'
###### S = 'Stride'
###### K = 'Kernel Size'
###### P = 'Padding'
###### output = ( I - 1 ) * S + K - 2 * P
###### input_size = 224,224 == 2
###### max_pool5 = 7,7 == 224/7 == 32x
###### max_pool4 = 14,14 == 224/14 == 16x
###### max_pool13 = 28,28 == 224/28 = 8x

In [23]:
## dummy input
model  = FCN8(num_classes=1000)
dummy_input=torch.rand(1,3,224,224)
output = model(dummy_input)
print('Output Shape: ',output.shape)

Output Shape:  torch.Size([1, 1000, 224, 224])
